In [ ]:
## Imports. This contains some imports not required for this notebook. 

import sys
import os
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import pylab                   # Needed to plot histograms.
sys.path.append('/Users/alex/mycode/DataPype') # Adds the path to the DataPype module to the system path.
from datapype.datafits import DataFits         # Gets the function that makes DataFits io objects.
from astropy.io import fits                    # Need this if you want to use astropy.io io objects.
from ipywidgets import interact                # Need this for interactive plots.
from matplotlib.colors import LogNorm          # Machinery for LogNorm scaling of intensities.
from matplotlib.colors import SymLogNorm       # Machinery for SymLogNorm scaling of intensities.
from matplotlib.colors import PowerNorm        # Machinery for LogNorm (e.g., square root) scaling of intensities.
from astropy.stats import mad_std              # The median absolute deviation, a more robust estimator than std.

'''The magic function in the following line ensures that plots are shown in the notebook and
permits use of matplotlib commands without prefixes (e.g., plot instead of plt.plot).
It is not the best python programming practice, but it does turn out to be convenient
for highly interactive work. See the following links for more information:
https://stackoverflow.com/questions/20961287/what-is-pylab for a quick explanation of what it does.
https://nbviewer.jupyter.org/github/Carreau/posts/blob/master/10-No-PyLab-Thanks.ipynb?create=1 for
a more extended discussion and an argument about why it is "better" not to use it.
'''
# %pylab inline

## The following adds a bunch of functions in a module stored in the path below.
#sys.path.append('/Users/alex/mycode/MyCode') # Adds the path to the MyCode package on my computer to the system path.
from dah_functions.dah_functions import head
from dah_functions.dah_functions import displaypic
from dah_functions.dah_functions import displaypic2
from dah_functions.dah_functions import show_image
from dah_functions.dah_functions import rowplot
from dah_functions.dah_functions import colplot
from dah_functions.dah_functions import rowcolplot

'''Make sure that plots are shown in the notebook'''
# %pylab inline

'''Don't need the following in this notebook, but may need for others. Hold on to them for now.'''
# import time                             # Various routines for time and data operations.
# import re                               # Regular expressions.
# import scipy.stats                      # Statistical routines.

'''Filter out warnings. May or may not need this. But shouldn't hurt to put it in.
This is the "new" code for avoiding warnings-- seems to work better. Copied from an SDSS notebook.'''
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Specify the path to the data directory.
# datapath = '/Users/alex/_observing/24-inch/2018/manual_dark/darks_180420'
datapath = 'C:/Users/Shamaul/Downloads/astr212/24inch_data/190421/dark/'

In [ ]:
## List files in the the data directory.

whichpath = datapath

## For ALL the files in the directory.
# allfiles = [f for f in os.listdir(datapath)]

## Various list comprehensions can pick out files with particular characteristics.
allfiles = [f for f in os.listdir(whichpath) if '.fit' in f and ('bias.' in f) and 'mdark' not in f]

allfiles = sorted(allfiles)       ## This is necessary on my Mac, may not be for others?
for i in range(len(allfiles)):
    print( i, allfiles[i])

In [ ]:
## Construct a list of files you wish to view and/or process.

acceptlist = True             # True if you want to accept all the files in allfiles.
contiguous = True             # True if you want to accept a contiguous subset of allfiles.
startfile, endfile = 0,9      # The first and last files in a contiquous subset.
flist = [0,3,4,5,6,7,8]        # An explicit list of the files you want to accept.

if acceptlist == True:
    files = allfiles
    for i in range(len(files)):
        print(i, files[i])
else:
    if contiguous == True:
        files = allfiles[startfile:endfile+1]
        for i in range(len(files)):
            print( i, files[i])
    else:
        files = []
        for i in range(len(flist)):
            files.append(allfiles[flist[i]])
        for i in range(len(files)):
            print( i, flist[i], files[i])

In [ ]:
## Read one file to determine the image shape (rows, cols).
## Also find the exposure time.

whichfile = 0
printheader = True
print('whichfile =',files[whichfile])
print('whichpath =',whichpath)
######################################################################
## Use this code if you are using astropy.io and want to get a list of the HDUs.
fitsfilename = os.path.join(whichpath,files[whichfile])
hdulist = fits.open(fitsfilename)   # Open the fits file as hdulist
oneimage = hdulist[0].data
rows, cols = oneimage.shape[0], oneimage.shape[1]
df = DataFits()                                    # Create a DataPype DataFits io object.
df.load(os.path.join(whichpath,files[whichfile]))   # Loads the file
print('rows =',rows,'  cols =',cols)
print('exptime = ', hdulist[0].header['exptime'])
whichfileheader = hdulist[0].header
######################################################################
# ## Use this code if you are using DataFits io.
# df = DataFits()                                    # Create a DataPype DataFits io object.
# df.load(os.path.join(whichpath,files[whichfile]))   # Loads the file.
# oneimage = df.imageget()
# rows, cols = oneimage.shape[0], oneimage.shape[1]
# print('rows =',rows,'  cols =',cols)
# print('exptime = ', df.header['exptime'])
# hdr = df.header
########################################################################

In [ ]:
'''Make a stack of images, a list of headers from those images, and calculate some medians and stds
that will help set autoscaling parameters.'''

## Assume that variables rows and cols are defined in cell above.
## Assume that whichpath is defined above.

image = np.zeros((len(files), rows,cols))  # 3D numpy array to hold the stack of images.
imedian = np.zeros((len(files)))           # 1D numpy array to hold array medians.
istd = np.zeros((len(files)))              # 1D numpy array to hold array stds.
madstd = np.zeros((len(files)))            # 1D numpy array to hold Median absolute deviations.

headlist = []                              # Empty list to hold the headers.
print('image.shape =', image.shape)

for i in range(len(files)):

#############################################################################
#     # Use this code block if you want to use DataFits io objects.
#     fitsfilename = os.path.join(whichpath, files[i])    # Full path to a fitsfile.
#     df = DataFits()
#     df.load(fitsfilename)                  # Open a fits file as a DataPype object.
#     image[i] = df.imageget() * 1.0                         # Define image in stack as float of data in the 0th hdu.
#     headlist.append(df.header)                       # Append the header of the fits object to the header list.
#     print('')
#     print(i,files[i])
################################################################################

#############################################################################
    # Use this code block if you want to use the standard astropy.io package.
    fitsfilename = os.path.join(whichpath, files[i])    # Full path to a fitsfile.
    hdulist = fits.open(fitsfilename)                  # Open a fits file as an hdulist object.
    hdu0 = hdulist[0]                                  # Define a fits object as the 0th hdu in the fitsfile.
    image[i] = hdu0.data * 1.0                         # Define image in stack as float of data in the 0th hdu.
    headlist.append(hdu0.header)                       # Append the header of the fits object to the header list.
    print('')
    print(i,files[i])
###############################################################################    
    
    ## Calculate some medians and stds to use in autoscaling that aren't unduly biased by extreme values.
    ## Optionally, print out masked and unmasked values to explore the effects of extreme values on the statistics.
    
    madstd[i] = mad_std(image[i],ignore_nan=True)
    imedian[i] = np.nanmedian(image[i])
    istd[i] = np.nanstd(image[i])
    print(imedian[i], istd[i], madstd[i], np.nanmin(image[i]), np.nanmax(image[i]))

    ## Now make some masks and exclude extreme values
    
    highmask = np.where(image[i] > (imedian[i] + 3 * madstd[i]))
    lowmask = np.where(image[i] < (imedian[i] - 3 * madstd[i]))
    
    above, below = len(highmask[0]), len(lowmask[0])

    maskedimage = image[i].copy()   # Make a copy to avoid modifying the original image data.
    maskedimage[highmask] = np.nan
    maskedimage[lowmask] = np.nan
    
    imedian_masked = np.nanmedian(maskedimage)
    istd_masked = np.nanstd(maskedimage)
    madstd_masked = mad_std(maskedimage,ignore_nan=True)
    print(imedian_masked, istd_masked, madstd_masked, np.nanmin(maskedimage), np.nanmax(maskedimage))
    
    print('pixels below =',below,'   pixels above =',above)

In [ ]:
bad = 0
for i in range(len(image)):
    istand = image[i] - imedian[i]
    bad += (istand > istd[i])
    bad += (istand < -istd[i])
for j in range(len(sum(bad))):
    print(str(j) + ':' + str(sum(bad)[j]))

In [ ]:

bads = np.zeros((1024,1024))

for i in range(len(image)):
    istand = image[i] - imedian[i]
    bad1 = (istand > istd[i]) * 1
    bad2 = (istand < -istd[i]) * 1
    bad = bad1 + bad2
    bads += bad
print(bads)
ctr = 0
for i in range(1024):
    for j in range(1024):
        if bads[i,j] >= 3:
            ctr += 1
print(str(ctr/(1024 * 1024)))

The above array gives the number of times, for each pixel, it has been deemed "bad" in the list of 30 darks.

If we check pixels that have been bad at least once: 49.2% of pixels meet this criteria If we check pixels that have been bad at least twice: 18.5% of pixels meet this criteria If we check pixels that have been bad at least 3 times: 8.28% of pixels meet this criteria If we check pixels that have been bad at least 4 times: 5.21% of pixels meet this criteria ...

In [ ]:
# # Display all the images in the stack.
# figx, figy = 20,20   # Use these to reveal fine detail in images.
figx, figy = 8,8   # Use these to get a quick look and save space.

for i in range(len(files)):
    plt.figure(figsize = (figx,figy))
    vmx= imedian[i] + madstd[i] * 3.0
    vmn = imedian[i] - madstd[i] * 3.0
#     grid()
    plt.title(files[i] + '   Median =' + str(imedian[i]) + '    mad_std =' + str(madstd[i]))
    plt.imshow(image[i],'gray',interpolation = 'nearest',vmax=vmx,vmin=vmn)

In [ ]:
# Make a median dark image from the image stack
dark = np.nanmedian(image, axis=0)
darkmed = np.nanmedian(dark)
darkstd = np.nanstd(dark)
darkmadstd = mad_std(dark,ignore_nan=True)

# Calculate the median and std of the median dark image constructed above.
# Exclude very large values from the calculation of the std (those greater than 2 times the median).
darkproxy = dark.copy()
darkmask = np.where(dark > darkmed * 2.0) # Criterion for eliminating very large pixel values from the std calculation.
darkproxy[darkmask] = np.nan
darkproxymed = np.nanmedian(darkproxy)
darkproxystd = np.nanstd(darkproxy)
darkproxymadstd = mad_std(darkproxy,ignore_nan=True)
print(darkmed, darkstd, darkmadstd)
print(darkproxymed, darkproxystd, darkproxymadstd)

In [ ]:
## What do the quantities listed in the following cell and their ratios tell us about
## the nature of the "noise" in the images? What would you expect to gain by taking the median
## over an even larger number of images?

print(imedian[0],istd[0],madstd[0])                 # Properties of one of the input images.
print(darkmed,darkstd,darkmadstd)                   # Properties of the median dark image.
print(darkproxymed, darkproxystd, darkproxymadstd)  # Properties of the masked median dark image.

In [ ]:
## Get one of the images in the stack and its name and header.
whichimage = 0

rawimage = image[whichimage].copy()
rawheader = headlist[whichimage]
rawname = files[whichimage] + 'xxx'
rawname = rawname[:-3]
rawmedian = np.nanmedian(rawimage)
rawmad = mad_std(rawimage,ignore_nan=True)
rawstd = np.nanstd(rawimage)
print('rawname =', rawname)
print('rawmedian =', rawmedian,'  rawstd=', rawstd,'  rawmad =', rawmad)
# rawheader

In [ ]:
#display the image
displaypic(rawimage,files[0],rows,cols,istart=(rawmedian-3*rawmad,rawmedian+10*rawmad),negative=True)

In [ ]:

#display a specific row and colum
rowcolplot(rawimage,files[whichimage],500,500,plotrow=True,plotcol=True)

In [ ]:
# List the dark images in the dark file directory.
# darkfiles = [f for f in os.listdir(darkpath) if '.fit' in f]                   # All the fits files.
# darkfiles = [f for f in os.listdir(darkpath) if '.fit' in f and 'dark' in f]    # All the dark files. 
darkfiles = [f for f in os.listdir(darkpath) if '.fit' in f and 'mdark' in f]   # All the median dark files.   


darkfiles = sorted(darkfiles)
for i in range(len(darkfiles)):
    print(i, darkfiles[i])

print('t')
# print('rawimage exposure time =', rawhdulist[0].header['exposure'])

In [ ]:
# useful for finding things above or below limits in darks

'''Get a bias image, calculate some statistics, make a masked image with "nans" replacing "bad" pixels, and
calculate some statistics on the masked image. In this case, "bad" is defined as pixels lying above or below
selected thresholds. For example, one might want to exclude "dead" pixels with no signal or "hot" pixels with
bias levels so high that that the pixel's dynamic range would be unacceptable.'''

whichfile = 1
highlimfactor = 20.
lowlimfactor = 2.

darkname = darkfiles[whichfile]    # Select a bias image.
dname = os.path.join(darkpath,darkname)
darkhdulist = fits.open(dname)
darkhdulist.info()
print('')

darkimage = darkhdulist[0].data * 1.0    # Read in the image and convert to float.
darkheader = darkhdulist[0].header       # Make a variable for the header of the dark.

# Calculate median and std of raw bias image.
darkmedian = np.nanmedian(darkimage)
darkstd = np.nanstd(darkimage)
darkmad = mad_std(darkimage)
darkmin = np.nanmin(darkimage)
darkmax = np.nanmax(darkimage)
print('darkmedian=', darkmedian, '  darkstd=', darkstd,'   darkmad=',darkmad, '  darkmin=', darkmin,'  darkmax=', darkmax)

# Set upper and lower thresholds for "good" pixels and make masks.
dupperlimit = highlimfactor * darkmedian   # Select an upper limit for "good" pixels.
dlowerlimit = darkmedian/lowlimfactor     # Select a lower limit for "good" pixels.
print('lower limit =',dlowerlimit, '  upper limit =', dupperlimit)
highdarkmask = np.where(darkimage > dupperlimit)
lowdarkmask = np.where(darkimage < dlowerlimit)

# Make masked images with 'nans' in masked pixels.
mdarkimage = darkimage.copy()
mdarkimage[highdarkmask] = np.nan
mdarkimage[lowdarkmask] = np.nan

# Re-compute median and std for masked image.
# NOTE: mad_std doesn't seem to work if there are nans in the image.
mdmedian = np.nanmedian(mdarkimage)
mdstd = np.nanstd(mdarkimage)
mdmad = mad_std(mdarkimage,ignore_nan=True)
mdmin = np.nanmin(mdarkimage)
mdmax = np.nanmax(mdarkimage)
print('mask median =',mdmedian,' mask std =',mdstd,' mask mad',mdmad,' mask min =',mdmin,' mask max =',mdmax)
print('low pixels =', len(lowdarkmask[0]),'   high pixels =', len(highdarkmask[0]),)

## Use this if you want to print the header.
# darkheader

In [ ]:
## List the low and/or high pixels in the darkmasks.

low = False
high = True
if low == True:
    print('Number of low pixels =',len(lowdarkmask[0]))
    print('')
    for i in range(len(lowdarkmask[0])):
        print(i, lowdarkmask[0][i], lowdarkmask[1][i])
if high == True:
    print('Number of high pixels =',len(highdarkmask[0]))
    print('')
    for i in range(len(highdarkmask[0])):
        print(i, highdarkmask[0][i], highdarkmask[1][i])

In [ ]:
#displaying the pic
displaypic(darkimage,darkname,rows,cols,istart=(darkmedian- 3* darkmad,darkmedian+3*darkmad),negative=False)

In [ ]:
## Plot some histograms of the images for diagnostic purposes.
## This uses histogram plotting routines from matplotlib.

NBINS = 200
plt.figure(figsize = ((18,8)))

print(darkname)
dlow, dhigh = np.nanmin(darkimage), np.nanmax(darkimage)
print( 'low =', dlow, ',   high =', dhigh)
dunderlimit, doverlimit = dlow, dupperlimit
dundermask, dovermask = np.where(darkimage < dunderlimit), np.where(darkimage >= doverlimit)

print( 'pixels <',dunderlimit, '=',len(dundermask[0]))
print( 'pixels >',doverlimit, '=', len(dovermask[0]))

plt.subplot(3,1,1)
plt.title(darkname)
# xlim(0000.,100)
#plt.ylim(0,200)
histogram = pylab.hist(darkimage.flatten(),NBINS,histtype='step',range=(dlow,doverlimit))
plt.subplot(3,1,2)
histogram = pylab.hist(darkimage.flatten(),log=True)

In [ ]:
## historgrams from compare_images
## Plot some histograms of the images for diagnostic purposes.
## This uses histogram plotting routines from matplotlib.

NBINS = 200
LOGBINS = 40
plt.figure(figsize = ((18,25)))

imagelist = ['image[0]','medianimage','npic[i]','noisepic','gimage']
plt.subplot(7,1,1)
plt.title('')
# xlim(0000.,100)
# ylim(0,200)
histogram = pylab.hist(image[0].flatten(),NBINS,histtype='step',range=(0.,2000.),color='b')
histogram = pylab.hist(medianimage.flatten(),NBINS,histtype='step',range=(0.,2000.),color='g')
histogram = pylab.hist(npic[0].flatten(),NBINS,histtype='step',range=(0.,2000.),color='c')
histogram = pylab.hist(noisepic.flatten(),NBINS,histtype='step',range=(0.,2000.),color='r')
histogram = pylab.hist(gimage.flatten(),NBINS,histtype='step',range=(0.,2000.),color='m')
histogram = pylab.hist(fimage.flatten(),NBINS,histtype='step',range=(0.,2000.),color='orange')

plt.subplot(7,1,2)
plt.title(files[0])
plt.grid()
histogram = pylab.hist(image[0].flatten(),LOGBINS,histtype = 'step',log=True,color='b')

plt.subplot(7,1,3)
plt.title('medianimage')
plt.grid()
histogram = pylab.hist(medianimage.flatten(),LOGBINS,histtype = 'step',log=True,color='g')

plt.subplot(7,1,4)
plt.title('npic[0]')
plt.grid()
histogram = pylab.hist(npic[0].flatten(),LOGBINS,histtype = 'step',log=True,color='c')

plt.subplot(7,1,5)
plt.title('noisepic')
plt.grid()
histogram = pylab.hist(noisepic.flatten(),LOGBINS,histtype = 'step',log=True,color='r')

plt.subplot(7,1,6)
plt.title('gimage')
plt.grid()
histogram = pylab.hist(gimage.flatten(),LOGBINS,histtype = 'step',log=True,color='orange')

plt.subplot(7,1,7)
plt.title('fimage')
plt.grid()
histogram = pylab.hist(fimage.flatten(),LOGBINS,histtype = 'step',log=True,color='m')

In [ ]:

# List the bias file directory.

# biasfiles = [f for f in os.listdir(biaspath) if '.fit' in f]
biasfiles = [f for f in os.listdir(biaspath) if '.fit' in f and 'mdark' in f and 'bias' in f]
biasfiles = sorted(biasfiles)
for i in range(len(biasfiles)):
    print(i, biasfiles[i])

In [ ]:
# Get a bias image, calculate some statistics, and make a masked image with "nans" replacing "bad" pixels.

biasname = biasfiles[0]    # Select a bias image.

hblimfactor = 2.
lblimfactor = 2.

bname = os.path.join(biaspath,biasname)
biashdulist = fits.open(bname)
biashdulist.info()
print('')

biasimage = biashdulist[0].data * 1.0    # Read in the image and convert to float.

print('')

# Calculate median and std of raw bias image.
biasmedian = np.nanmedian(biasimage)
biasstd = np.nanstd(biasimage)
biasmad = mad_std(biasimage,ignore_nan=True)
biasmin = np.nanmin(biasimage)
biasmax = np.nanmax(biasimage)
print('median=', biasmedian, '  std=', biasstd, '  min=', np.nanmin(biasimage),'  max=', np.nanmax(biasimage))

# Set upper and lower thresholds for "good" pixels and make masks.
bupperlimit = hblimfactor * biasmedian   # Select an upper limit for "good" pixels.
blowerlimit = biasmedian/lblimfactor     # Select a lower limit for "good" pixels.
print('lower limit =',blowerlimit, '  upper limit =', bupperlimit)
highbiasmask = np.where(biasimage > bupperlimit)
lowbiasmask = np.where(biasimage < blowerlimit)
print('high pixels =', len(highbiasmask[0]), '    low pixels =', len(lowbiasmask[0]))

# Make masked images with 'nans' in masked pixels.
mbiasimage = biasimage.copy()
mbiasimage[highbiasmask] = np.nan
mbiasimage[lowbiasmask] = np.nan

# Re-compute median and std for masked image.
mbmedian = np.nanmedian(mbiasimage)
mbstd = np.nanstd(mbiasimage)
mbmad = mad_std(mbiasimage,ignore_nan=True)
mbmin = np.nanmin(mbiasimage)
mbmax = np.nanmax(mbiasimage)
print('mask median =',mbmedian,' mask std =', mbstd,'mask mad =',mbmad, ' mask min =',mbmin,' masked max =',mbmax)

In [ ]:
## List the low and/or high pixels in the biasmasks.

low = True
high = True
if low == True:
    print('Number of low pixels =',len(lowbiasmask[0]))
    for i in range(len(lowbiasmask[0])):
        print(i, lowbiasmask[0][i], lowbiasmask[1][i])
    print('')
if high == True:
    print('Number of high pixels =',len(highbiasmask[0]))
    for i in range(len(highbiasmask[0])):
        print(i, highbiasmask[0][i], highbiasmask[1][i])

In [ ]:
## Make list of high pixels (greater than some value) in the biasimage and their values. DIAGNOSTIC.

hbmask = np.where(biasimage > bupperlimit)
print( hbmask)
# print hbmask[0][0]
for i in range(len(hbmask[0])):
    print ('{:3}{:4}{:5}{:10}'.format(i,hbmask[0][i],hbmask[1][i],biasimage[hbmask[0][i],hbmask[1][i]]))

In [ ]:
# Make a dark current image. Also make masks to screen out high and low pixels. 

dcimage = darkimage - biasimage
dcheader = darkheader.copy()
dcheader['proctype'] = 'dark current'
dcheader['filelist'] = darkname + ', ' + biasname

# Calculate some statistics

dcmin, dcmax = np.nanmin(dcimage), np.nanmax(dcimage)
print(dcmin, dcmax)
dcmedian = np.nanmedian(dcimage)
dcstd = np.nanstd(dcimage)
dcmad = mad_std(dcimage)
print(dcmedian, dcstd, dcmad)

dclowerlimit = dcmedian - 5 * dcmad
dcupperlimit = dcmedian + 100 * dcmad
# dclowerlimit = dcmedian - 5 * dcmad
# dcupperlimit = dcmedian + 50 * dcmad
lowdcmask = np.where(dcimage <  dclowerlimit)
highdcmask = np.where(dcimage > dcupperlimit)

print('high pixels =', len(highdcmask[0]), '    low pixels =', len(lowdcmask[0]))

mdcimage = dcimage.copy()
mdcimage[highdcmask] = np.nan
mdcimage[lowdcmask] = np.nan

mdcmedian = np.nanmedian(mdcimage)
mdcstd = np.nanstd(mdcimage)
print(mdcmedian, mdcstd, np.max(mdcimage), np.min(mdcimage))

In [ ]:
## Print lists of pixels above or below some threshold.
printlist = 'no'

print('max pixel =',np.max(dcimage))
himask = np.where(dcimage > dcupperlimit)  # Or, put in some other limit.
lomask = np.where(dcimage < dclowerlimit)  # Or, put in some other limit.

whichmask = himask

print('pixels above threshold =', len(whichmask[0]), '    threshold =', dcupperlimit)
print('')

if printlist == 'yes':
    for i in range(len(whichmask[0])):
        print ('{:3}{:5}{:5}{:10}'.format(i,whichmask[0][i],whichmask[1][i],dcimage[whichmask[0][i],whichmask[1][i]]))

In [ ]:
# Get a flat image, calculate some statistics, and make a masked image with "nans" replacing "bad" pixels.

whichfile = 0

hflimfactor = 2.
lflimfactor = 2.

flatname = flatfiles[whichfile]    # Select a flat image.
fname = os.path.join(flatpath,flatname)
flathdulist = fits.open(fname)
flathdulist.info()
print('')

flatimage = flathdulist[0].data * 1.0    # Read in the image and convert to float.
flatheader = flathdulist[0].header.copy()

print('')

# Calculate median and std of raw flat image.
flatmedian = np.nanmedian(flatimage)
flatstd = np.nanstd(flatimage)
flatmad = mad_std(flatimage,ignore_nan=True)
flatmin = np.nanmin(flatimage)
flatmax = np.nanmax(flatimage)
print('median=',flatmedian,'  std=',flatstd,'flatmad =',flatmad,'  min=',flatmin,'  max=', flatmax)

# Set upper and lower thresholds for "good" pixels and make masks.
fupperlimit = hflimfactor * flatmedian   # Select an upper limit for "good" pixels.
flowerlimit = flatmedian/lflimfactor     # Select a lower limit for "good" pixels.
print('lower limit =',flowerlimit, '  upper limit =', fupperlimit)
highflatmask = np.where(flatimage > fupperlimit)
lowflatmask = np.where(flatimage < flowerlimit)
print('high pixels =', len(highflatmask[0]), '    low pixels =', len(lowflatmask[0]))

# Make masked images with 'nans' in masked pixels.
mflatimage = flatimage.copy()
mflatimage[highflatmask] = np.nan
mflatimage[lowflatmask] = np.nan

# plt.colorbar()
print('highmask =',highflatmask)

# Re-compute median and std for masked image.
mfmedian = np.nanmedian(mflatimage)
mfstd = np.nanstd(mflatimage)
mfmad = mad_std(mflatimage,ignore_nan=True)
mfmin = np.nanmin(mflatimage)
mfmax = np.nanmax(mflatimage)
print('mask median =',mfmedian,' mask std =', mfstd,'mask mad =',mfmad, ' mask min =',mfmin,' mask max =',mfmax)

make bad pixels for a flat image

In [ ]:
# ## Make lists of high and low pixels (more or less than 5 sigma) in the flatimage ("suspicious pixels") and their values

# # Print lists?
printlists = 'no'

multiplier = 10.

highlimit = flatmedian + multiplier*flatmad
lowlimit = flatmedian - multiplier*flatmad
hfmask = np.where(flatimage > highlimit)   # Or put in some other value.
lfmask = np.where(flatimage < lowlimit)   # Or put in some other value.
print('high threshold =',highlimit,'   low threshold =',lowlimit)
print( 'high pixels =', len(hfmask[0]), '    low pixels =', len(lfmask[0]))
print( '')
if printlists == 'yes':
    print( 'list of high pixels')
    for i in range(len(hfmask[0])):
        print( ('{:3}{:5}{:5}{:>10.3f}'.format(i,hfmask[0][i],hfmask[1][i],flatimage[hfmask[0][i],hfmask[1][i]])))
    print( '')
    print( 'low pixels')
    for i in range(len(lfmask[0])):
        print( ('{:3}{:5}{:5}{:>10.3f}'.format(i,lfmask[0][i],lfmask[1][i],flatimage[lfmask[0][i],lfmask[1][i]])))

In [ ]:
#plot a row and column
whichimage = 0
rowcolplot(image[whichimage],files[whichimage],500,500,autoscale=True,ylims=(0., 65535.),xstep=100,ystep=100)

In [ ]:
'''Take the difference between two images and blink between one and the difference image.'''
A, B = 0,1
nameA = files[A]
nameB = files[B]
imageA = image[A]
imageB = image[B]
difimage = imageA - imageB
difname = 'difference image'

# Compare the 
blink2images(imageA,nameA,difimage,difname,linthresh=100.,linscale=0.3,lims=(-10.,65535.,1.),addgrid=False,xystep=1)

In [ ]:
# make a median image
# Make a median image from the image stack
medianimage = np.nanmedian(image, axis=0)
medim_med = np.nanmedian(medianimage)
medim_std = np.nanstd(medianimage)
medim_madstd = mad_std(medianimage,ignore_nan=True)

## Diagnostics:
## Exclude very large values from the calculation of the std (those greater than 2 times the median).
medimproxy = medianimage.copy()
highmask = np.where(medianimage > medim_med * 2.0) # Criterion for eliminating very large pixel values from the std calculation.
medimproxy[highmask] = np.nan
medimproxymed = np.nanmedian(medimproxy)
medimproxystd = np.nanstd(medimproxy)
medimproxymadstd = mad_std(medimproxy,ignore_nan=True)
print(medim_med, medim_std, medim_madstd)
print(medimproxymed, medimproxystd, medimproxymadstd)

In [ ]:
#may or my not be useful
# Calculate median and std of raw flat image.
flatmedian = np.nanmedian(flatimage)
flatstd = np.nanstd(flatimage)
flatmad = mad_std(flatimage,ignore_nan=True)
flatmin = np.nanmin(flatimage)
flatmax = np.nanmax(flatimage)
print('median=',flatmedian,'  std=',flatstd,'flatmad =',flatmad,'  min=',flatmin,'  max=', flatmax)

# Set upper and lower thresholds for "good" pixels and make masks.
fupperlimit = hflimfactor * flatmedian   # Select an upper limit for "good" pixels.
flowerlimit = flatmedian/lflimfactor     # Select a lower limit for "good" pixels.
print('lower limit =',flowerlimit, '  upper limit =', fupperlimit)
highflatmask = np.where(flatimage > fupperlimit)
lowflatmask = np.where(flatimage < flowerlimit)
print('high pixels =', len(highflatmask[0]), '    low pixels =', len(lowflatmask[0]))

# Make masked images with 'nans' in masked pixels.
mflatimage = flatimage.copy()
mflatimage[highflatmask] = np.nan
mflatimage[lowflatmask] = np.nan

# Re-compute median and std for masked image.
mfmedian = np.nanmedian(mflatimage)
mfstd = np.nanstd(mflatimage)
mfmad = mad_std(mflatimage,ignore_nan=True)
mfmin = np.nanmin(mflatimage)
mfmax = np.nanmax(mflatimage)
print('mask median =',mfmedian,' mask std =', mfstd,'mask mad =',mfmad, ' mask min =',mfmin,' mask max =',mfmax)

In [ ]:
'''Filter an image and compare with the original. The idea here is that the filtered image will
substitute a valid value for wildly discrepant pixels and the differenced image will make the bad
pixels stand out.'''

# inimage = image[0][2400:2500,3675:3800].copy()
inimage = noisepic.copy()
imagename = 'noisepic'
fimage = nd.median_filter(inimage,size=3)
fname = 'median filtered image'
gimage = nd.gaussian_filter(fimage,sigma=3)
gname = 'gaussian filter on top of median filter'
difimage = inimage - fimage * 1.0
difname = 'difference image'

# Compare them:
# blink2images(inimage,imagename,difimage,difname,linthresh=100.,linscale=0.3,lims=(-10.,10.,.01),addgrid=True,xystep=10)
blink2images(inimage,imagename,gimage,gname,linthresh=100.,linscale=0.3,lims=(-10.,10.,.01),addgrid=False,xystep=10)

In [ ]:
## Make lists of high and low pixels (more or less than 5 sigma) in the flatimage ("suspicious pixels") and their values

# Print lists?
printlists = 'no'

multiplier = 10.

highlimit = flatmedian + multiplier*flatmad
lowlimit = flatmedian - multiplier*flatmad
hfmask = np.where(flatimage > highlimit)   # Or put in some other value.
lfmask = np.where(flatimage < lowlimit)   # Or put in some other value.
print('high threshold =',highlimit,'   low threshold =',lowlimit)
print( 'high pixels =', len(hfmask[0]), '    low pixels =', len(lfmask[0]))
print( '')
if printlists == 'yes':
    print( 'list of high pixels')
    for i in range(len(hfmask[0])):
        print( ('{:3}{:5}{:5}{:>10.3f}'.format(i,hfmask[0][i],hfmask[1][i],flatimage[hfmask[0][i],hfmask[1][i]])))
    print( '')
    print( 'low pixels')
    for i in range(len(lfmask[0])):
        print( ('{:3}{:5}{:5}{:>10.3f}'.format(i,lfmask[0][i],lfmask[1][i],flatimage[lfmask[0][i],lfmask[1][i]])))

In [ ]:
'''Use scipy.ndimage.labels() to label regions with intensities higher than some value.
Print out for each region the centers of mass, the maximum pixel value, the
sum of the pixel values, the ratio of the sum to the max, and the number of pixels in the region.'''

threshold = 1000

# sub = maskedimage[500:600,25:75]
difmask = np.where(difimage < threshold)
difcopy = difimage.copy()
difcopy[difmask] = 0.
sub = difcopy

difmask100 = np.where(difimage[2400:2500,3675:3800] > threshold)   # This is a test to see what the mask looks like.

# plt.imshow(sub,'gray_r', vmin = 0, vmax = 1)
labels = nd.label(sub)
print( '')

# Find how many areas:
areas = np.max(labels[0])   # The number of areas.
print( areas)
print( '')
# Find out how many pixels are in each labeled area.
pixperarea = np.zeros((areas+1))
print( '')
for i in range(1,areas+1):
    pixperarea[i] = nd.sum(labels[0],labels[0], index = i)/i


print( '')    

lpic = labels[0] * 1.
plt.figure(figsize = (8,8))
plt.imshow(lpic[2400:2500,3675:3800],'gray_r', vmin = 0, vmax = 0.1, interpolation = 'nearest')

plt.figure(figsize = (8,8))
plt.imshow(difimage[2400:2500,3675:3800],'gray_r', interpolation = 'nearest')
plt.colorbar()

plt.figure(figsize = (8,8))
plt.imshow(image[5][2400:2500,3675:3800],'gray_r', interpolation = 'nearest')
plt.colorbar()


# print labels[0]
# test = np.where(maskedimage != 0.)
# print test[0:10]

# for i in range(1,np.max(labels[0])+1):

# Print the whole list
for i in range(1,areas+1):
# for i in range(1,5):       # Diagnostic.
    com = nd.center_of_mass(sub, labels[0], index = i)
#     ext = nd.extrema(sub,labels[0], index = i)      # This one returns the maximum and minimum pixel in the region.
    picmax = nd.maximum(sub,labels[0], index = i)
    picsum = nd.sum(sub,labels[0], index = i)
    ratio = picsum/picmax
#     print i, com[0],com[1], '   ',picsum, '    ', ratio
    print ('{:<5}{:9.2f}{:11.2f}{:11.2f}{:11.2f}{:11.2f}{:11.2f}'.format(i,com[0],com[1],picmax, picsum,ratio, pixperarea[i]))
    
    
# print com2
# print type(labels[0])
# print labels[0].shape
# for i in range(len(labels)):
#     print i, labels[i]
# coms = nd.center_of_mass(maskedimage,labels[1],[1,2])
# print coms

# ## Just print out the areas with number of pixels per region > 2.
# print ''
# n = 0
# for i in range(1,areas+1):
#     com = nd.center_of_mass(sub, labels[0], index = i)
# #     ext = nd.extrema(sub,labels[0], index = i)
#     picmax = nd.maximum(sub,labels[0], index = i)
#     picsum = nd.sum(sub,labels[0], index = i)
#     ratio = picsum/picmax
#     if picmax > 100 and pixperarea[i] > 2.0:
#         n = n+1
#         print ('{:<5}{:9.2f}{:11.2f}{:11.2f}{:11.2f}{:11.2f}{:11.2f}'.format(i,com[0],com[1],picmax, picsum,ratio, pixperarea[i]))
# print ''
# print 'n =',n

In [ ]:
#making a masked flat
mflatimage = flatimage.copy()
mflatimage[hfmask]= np.nan
mflatimage[lfmask]= np.nan
over, under = len(hfmask), len(lfmask)
print('pixels under =', under,'   pixels over=',over)
mfmedian = np.nanmedian(mflatimage)
mfstd = np.nanstd(mflatimage)
mfmad = mad_std(mflatimage,ignore_nan=True)
print( mfmedian, mfstd, mfmad)
mflatimage = mflatimage / mfmedian
mfmedian = np.nanmedian(mflatimage)
mfstd = np.nanstd(mflatimage)
print( mfmedian, mfstd)

In [ ]:
# Fill the masked pixels with the masked array median
nanmask = np.where(np.isnan(mflatimage))
mflatimage[nanmask] = mfmedian